In [ ]:
##ref:https://www.kaggle.com/appian/panda-imagehash-to-detect-duplicate-images 

I think there is a lot overlap between the training data from the previous petfinder competition and the training data from this time.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os,cv2,tqdm,glob
import cv2,sys
import imagehash
from tqdm import tqdm_notebook as tqdm
from PIL import Image

funcs = [
    imagehash.average_hash,
    imagehash.phash,
    imagehash.dhash,
    imagehash.whash,
    #lambda x: imagehash.whash(x, mode='db4'),
]

In [ ]:
new_tra = glob.glob("../input/petfinder-pawpularity-score/train/*jpg")
old_tra = glob.glob("../input/petfinder-adoption-prediction/train_images/*-1.jpg")
old_test = glob.glob("../input/petfinder-adoption-prediction/test_images/*-1.jpg")
paths = new_tra+old_tra#+old_test
print(len(new_tra),len(old_tra),len(old_test))

In [ ]:
hashes = []
for path in tqdm(paths, total=len(paths)):

    image = cv2.imread(path)
    image = Image.fromarray(image)
    hashes.append(np.array([f(image).hash for f in funcs]).reshape(256))

In [ ]:
import torch

hashes = torch.Tensor(np.array(hashes).astype(int)).cuda()

In [ ]:
sims = np.array([(hashes[i] == hashes).sum(dim=1).cpu().numpy()/256 for i in range(hashes.shape[0])])
print(sims.shape)

In [ ]:
import matplotlib.pyplot as plt

threshold = 0.9
duplicates = np.where(sims > threshold)

cnt = 0
for i,j in zip(*duplicates):
    if i == j:
        continue
    cnt+=1
print("DUPLICATE",cnt)#3652
    

sys.exit()

pairs = {}
for i,j in zip(*duplicates):
    if i == j:
        continue

    path1 = paths[i]
    path2 = paths[j]
    print(path1)
    print(path2)

    image1 = cv2.imread(path1)
    image2 = cv2.imread(path2)
    
    if image1.shape[0] > image1.shape[1] / 2:
        fig,ax = plt.subplots(figsize=(20,20), ncols=2)
    elif image1.shape[1] > image1.shape[0] / 2:
        fig,ax = plt.subplots(figsize=(20,20), nrows=2)
    else:
        fig,ax = plt.subplots(figsize=(20,30), nrows=2)
    ax[0].imshow(image1)
    ax[1].imshow(image2)
    plt.show()

In [ ]:
del hashes

In [ ]:
new_tra_old_test = new_tra+old_test
hashes_1 = []
for path in tqdm(new_tra_old_test, total=len(new_tra_old_test)):

    image = cv2.imread(path)
    image = Image.fromarray(image)
    hashes_1.append(np.array([f(image).hash for f in funcs]).reshape(256))

In [ ]:
hashes_1 = torch.Tensor(np.array(hashes_1).astype(int)).cuda()

In [ ]:
sims1 = np.array([(hashes_1[i] == hashes_1).sum(dim=1).cpu().numpy()/256 for i in range(hashes_1.shape[0])])
print(sims1.shape)

In [ ]:
threshold = 0.9
duplicates1 = np.where(sims1 > threshold)

cnt1 = 0
for i,j in zip(*duplicates1):
    if i == j:
        continue
    cnt1+=1
print("DUPLICATE",cnt1)#

In [ ]:
del hashes_1

In [ ]:
hashes_2 = []
for path in tqdm(new_tra, total=len(new_tra)):

    image = cv2.imread(path)
    image = Image.fromarray(image)
    hashes_2.append(np.array([f(image).hash for f in funcs]).reshape(256))
    
hashes_2 = torch.Tensor(np.array(hashes_2).astype(int)).cuda()
sims2 = np.array([(hashes_2[i] == hashes_2).sum(dim=1).cpu().numpy()/256 for i in range(hashes_2.shape[0])])
print(sims2.shape)

In [ ]:
threshold = 0.9
duplicates2 = np.where(sims2 > threshold)

cnt2 = 0
for i,j in zip(*duplicates2):
    if i == j:
        continue
    cnt2+=1
print("DUPLICATE",cnt2)#